In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

columnNames = ['SepalLength', 'SepalWidth', 'PetalLength', "PetalWidth", 'Species']
species = ["Setosa", "Versicolor", "Virginica"]

trainPath = tf.keras.utils.get_file('iris_training.csv', "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
testPath = tf.keras.utils.get_file('iris_test.csv', "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(trainPath, names=columnNames, header = 0)
test = pd.read_csv(testPath, names=columnNames, header = 0)

# get rid of the species column, keep in seperate list
trainKeys = train.pop('Species')
testKeys = test.pop('Species')

def input_fn(features, labels, training=True, batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    if training:
        dataset=dataset.shuffle(1000).repeat()
        
    return dataset.batch(batch_size)

featureColumns = []
for key in train.keys():
    featureColumns.append(tf.feature_column.numeric_column(key = key))

# building the model
# different models work, there is DNN (deep neural nework)
# there is also linear classifiers.

classifier = tf.estimator.DNNClassifier(
    feature_columns = featureColumns,
    # two hidden layers of 30 and 10 nodes
    hidden_units=[30,10],
    n_classes=3
)

# train the model
classifier.train(
    input_fn = lambda: input_fn(train, trainKeys, training=True),
    # we will go thru the dataset until 5000 things have been looked at
    steps = 5000
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\owenm\\AppData\\Local\\Temp\\tmpx6qbav_u', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_v

In [2]:
# EVALUATE THE RESULT

eval_result = classifier.evaluate(
    input_fn = lambda: input_fn(test, testKeys, training=False)
)

print(eval_result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-20T12:10:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\owenm\AppData\Local\Temp\tmpx6qbav_u\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.32117s
INFO:tensorflow:Finished evaluation at 2021-11-20-12:10:22
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7, average_loss = 0.5865473, global_step = 5000, loss = 0.5865473
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\owenm\AppData\Local\Temp\tmpx6qbav_u\model.ckpt-5000
{'accuracy': 0.7, 'average_loss': 0.5865473, 'loss': 0.5865473, 'global_step': 5000}


In [12]:
def input_fn(features, batch_size = 256):
    return(tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size))

features = ['SepalLength', "SepalWidth", "PetalLength", "PetalWidth"]
predict = {}

print('please type numeric values as prompted')
for feature in features:
    invalid = True
    val = input(feature + ": ")
    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))

for prediction in predictions:
    print(prediction)
    classId = prediction['class_ids'][0]
    probability = prediction['probabilities'][classId]
    
    print('prediction is ', species[classId], 100*probability)

please type numeric values as prompted


ValueError: could not convert string to float: ''